# 在线程中使用Lock来防止数据竞争

In [1]:
import logging
from pprint import pprint

**示例：**该程序要从一整套传感器网络中对光照级别进行采样，采集到的样本总数，随着程序的运行不断增多。

In [2]:
class Counter(object):
    def __init__(self):
        self.count = 0

    def increment(self, offset):
        self.count += offset

在查询传感器读数的过程中，会发生阻塞式I/O操作。要给每个传感器分配它自己的工作线程。每采集到一次读数，工作线程就给Counter对象的value值加1，然后继续采集，直到完成全部的采样操作。

In [3]:
def worker(sensor_index, how_many, counter):
    # I have a barrier in here so the workers synchronize
    # when they start counting, otherwise it's hard to get a race
    # because the overhead of starting a thread is high.
    BARRIER.wait()
    for _ in range(how_many):
        # Read from the sensor
        counter.increment(1)

In [4]:
from threading import Barrier, Thread
BARRIER = Barrier(5)
def run_threads(func, how_many, counter):
    threads = []
    for i in range(5):
        args = (i, how_many, counter)
        thread = Thread(target=func, args=args)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()

In [5]:
how_many = 10**5
counter = Counter()
run_threads(worker, how_many, counter)
print('Counter should be %d, found %d' %
      (5 * how_many, counter.count))

Counter should be 500000, found 261710


**问题：**输出的结果与正确的结果相差很远。

**解决方法：**为了防止数据竞争行为，采用threading模块的Lock类，保护自己的数据结构不受破坏。

In [6]:
from threading import Lock

class LockingCounter(object):
    def __init__(self):
        self.lock = Lock()
        self.count = 0

    def increment(self, offset):
        with self.lock:
            self.count += offset

In [7]:
BARRIER = Barrier(5)
counter = LockingCounter()
run_threads(worker, how_many, counter)
print('Counter should be %d, found %d' %
      (5 * how_many, counter.count))

Counter should be 500000, found 500000
